In [ ]:
# Import 
import os
import time
import json
import datetime
import sys

import pandas as pd
import numpy as np
from matplotlib import rcParams

from pathlib import Path
from tyssue import Sheet
from tyssue.io import hdf5

from tyssue.dynamics.sheet_gradients import height_grad
from tyssue.dynamics import units, effectors, model_factory
from tyssue.solvers.quasistatic import QSSolver
from tyssue.draw import sheet_view
import matplotlib.pyplot as plt
%matplotlib inline
rcParams['figure.dpi'] = 200

sys.path.append("..")
from peripodial.polarity import model
from peripodial.dynamics import EllipsoidLameGeometry as geom
from peripodial.anisotropy import anysotropy as aniso
from peripodial.anisotropy import face_orientation
from peripodial.polarity import update_weights
from peripodial.draw import draw_half_sheet

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
SIM_DIR = Path('/home/lglepin/Simulations/')
today = datetime.date.today()

sim_save_dir = SIM_DIR/f'{today.isoformat()}_YolkElasticity'

try:
    sim_save_dir.mkdir(parents=True, exist_ok=True)
except FileExistsError:
    pass

dsets = hdf5.load_datasets('../json_hdf5/polarity.hdf5',
                               data_names=['vert', 'edge', 'face'])

with open('../json_hdf5/polarity.json', 'r+') as fp:
    specs = json.load(fp)

sheet = Sheet('spherical', dsets)

sheet.update_specs(specs)
geom.update_all(sheet)


In [ ]:
fig, ax = sheet_view(sheet, coords=["z","x"])

In [ ]:
aniso_cell_values = [] 
lumen_vol_egg= []
theta = np.pi / 3
be = np.arange(1, 300, 20)

for i, be in enumerate(bts):
    sheet_copy=sheet.copy()
    print(f"{i+1} / {10}", end=" ")
    update_weights(sheet_copy, threshold_angle=theta, below_threshold=1, above_threshold=6.75,
    )
    sheet.settings['barrier_elasticity']=be
    geom.update_all(sheet)
    solver = QSSolver(with_t1=True, with_t3=False)
    res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-4})
    sheet_copy.face_df['anisotropy'] = aniso(sheet_copy, coords = ['x','y','z'])
    aniso_cell = sheet_copy.face_df['anisotropy'].mean()
    aniso_cell_values.append(aniso_cell)
    lumen_vol_egg.append(sheet_copy.settings['lumen_vol'])

aniso_cell_values = np.array(aniso_cell_values)
print(aniso_cell_values)


In [ ]:
sheets_obj = {}
for i, above_threshold in enumerate(be):
    
    print(f"{i+1} / {10}", end=" ")
    update_weights(sheet, threshold_angle=theta, below_threshold=0.5, above_threshold=6.75,
    )
    solver = QSSolver(with_t1=True, with_t3=False)
    res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-4})
    sheet_copy = sheet.copy()
    sheets_obj[above_threshold] = sheet_copy



In [ ]:
fig, axes = plt.subplots(1, 10)

for i, (above_threshold, sheet_copy) in enumerate(sheets_obj.items()):
    #plt.savefig(f'sheet {i+1}.png', transparent=True)
    fig, ax = draw_half_sheet(sheet_copy, axes[i])
    ax.set_xlabel(f"{above_threshold:.3f}")
    ax.set_facecolor("black")
fig.set_size_inches(18, 5)
plt.show
plt.savefig('../image/aniso_weight.png', transparent=False)